In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from numbasom.core import *

# NumbaSOM

> A Fast Self-Organizing Map Python Library Implemented in Numba.

If you need a **fast and simple to use** SOM library implemented as a 2D lattice or torus, check this out. It utilizes online rather than batch training. 

## Install

`pip install numbasom`

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
import numpy as np

In [ ]:
data = np.random.randn(100,3)

In [ ]:
som = NumbaSOM((20,20),1000,data)

hello


In [ ]:
som.say_hello("Nikola")

'Hello Nikola!'

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()